# Adversarial Attack Generator

Based on: https://github.com/Trusted-AI/adversarial-robustness-toolbox/wiki/Get-Started#setup </p>
- Docs: https://adversarial-robustness-toolbox.readthedocs.io/en/latest/index.html


---

## Load Data Split

In [20]:
import pandas as pd

X_train = pd.read_csv("../CICIDS2017/train_test_split/X_train_proto.csv")
X_test = pd.read_csv("../CICIDS2017/train_test_split/X_test_proto.csv")
y_train = pd.read_csv("../CICIDS2017/train_test_split/y_train_proto.csv")
y_test = pd.read_csv("../CICIDS2017/train_test_split/y_test_proto.csv")

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(8000, 70) (2000, 70) (8000, 2) (2000, 2)


## Import Model

In [21]:
# load model
from tensorflow import keras

model = keras.models.load_model('../CICIDS2017/models/ids_dnn.keras')
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 50)             │         3,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 30)             │         1,530 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 10)             │           310 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 2)              │            22 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,238 (63.43 KB)

 Trainable params: 5,412 (21.14 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 10,826 (42.29 KB)

## Convert Model

In [22]:
# convert model to ART -> needed for adversarial attacks
from art.estimators.classification import TensorFlowV2Classifier
import tensorflow as tf

# Define loss function
loss_object = keras.losses.BinaryCrossentropy()
optimizer = keras.optimizers.Adam(learning_rate=0.001)
input_dim = X_train.shape[1] 

@tf.function
def custom_train_step(model, x_batch, y_batch):
    with tf.GradientTape() as tape:
        predictions = model(x_batch, training=True)
        loss = loss_object(y_batch, predictions)
    
    # Compute and apply gradients
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    return loss

# KerasClassifier uses tf.keras.backend.placeholder, which has been removed in TensorFlow 2.10+.so we need to use TensorFlowV2Classifier
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=2,  # Binary classification (0 or 1)
    input_shape=(input_dim,),  # Input shape
    clip_values=(0, 1), # because of the min-max normalization
    optimizer=optimizer, 
    loss_object=loss_object,
    train_step=custom_train_step  # Use default training function
)

# print accuracy
from sklearn.metrics import classification_report, accuracy_score

y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

print(classification_report(y_test, y_pred, target_names=['BENIGN', 'ATTACK']))
print(f"Accuracy : {accuracy_score(y_test, y_pred)*100:.2f}%")

              precision    recall  f1-score   support

      BENIGN       0.99      0.99      0.99      1619
      ATTACK       0.94      0.97      0.95       381

   micro avg       0.98      0.98      0.98      2000
   macro avg       0.97      0.98      0.97      2000
weighted avg       0.98      0.98      0.98      2000
 samples avg       0.98      0.98      0.98      2000

Accuracy : 98.20%


## Generate Attacks

In [23]:
# convert to numpy -> needed for adversarial attacks
X_test_np = X_test.to_numpy()

### Carlini & Wagner Attack - White Box

In [9]:
# from art.attacks.evasion import CarliniL2Method

# # Create the C&W attack (non-targeted)
# attack_cw = CarliniL2Method(classifier=classifier, confidence=0.0, targeted=False)

# # Generate adversarial examples on the test set
# X_test_adv_cw = attack_cw.generate(x=X_test_np)
# print(f'Adversarial C&W examples generated. Shape: {X_test_adv_cw.shape}')

C&W L_2:   0%|          | 0/500 [00:00<?, ?it/s]

Adversarial examples generated. Shape: (500, 70)


### FGSM Attack - White Box

In [24]:
from art.attacks.evasion import FastGradientMethod

# Create FGSM attack
attack_fgsm = FastGradientMethod(estimator=classifier, eps=0.5)  # ε tune this for stronger/weaker attacks: 0.01 weak, 0.1 balanced, 0.3-0.5 strong, 1 very strong
# the higher the epsilon, the easier it will be detected

# Generate adversarial examples
X_test_adv_fgsm = attack_fgsm.generate(x=X_test_np)
print(f'Adversarial FGSM examples generated. Shape: {X_test_adv_fgsm.shape}')

Adversarial FGSM examples generated. Shape: (2000, 70)


### HopSkipJumpAttack - Black Box

In [25]:
from art.attacks.evasion import HopSkipJump
import numpy as np

# Create HopSkipJump attack
attack_hop_skip_jump = HopSkipJump(classifier=classifier, targeted=False, norm=2)

# create target labels: [1, 0]
target_labels = np.ones((X_test_np.shape[0], 2))
target_labels[:, 0] = 1
target_labels[:, 1] = 0
print(target_labels[:2])


# Generate adversarial examples
X_test_adv_hop_skip_jump = attack_hop_skip_jump.generate(x=X_test_np)
print(f'Adversarial HopSkipJump examples generated. Shape: {X_test_adv_hop_skip_jump.shape}')

[[1. 0.]
 [1. 0.]]


HopSkipJump:   0%|          | 0/2000 [00:00<?, ?it/s]

Adversarial HopSkipJump examples generated. Shape: (2000, 70)


## Evaluate Attacks

In [8]:
# check for negative values in attacks
print(np.any(X_test_adv_fgsm < 0))
print(np.any(X_test_adv_hop_skip_jump < 0))

False
False


In [26]:
# Evaluate the model on adversarial examples
loss, accuracy = model.evaluate(X_test_np, y_test)
print(f'Accuracy on clean examples: {accuracy * 100:.2f}%')

#loss_adv, accuracy_adv = model.evaluate(X_test_adv_cw, y_test)
#print(f'Accuracy on c&w attack: {accuracy_adv * 100:.2f}%')

loss_adv_fgsm, accuracy_adv_fgsm = model.evaluate(X_test_adv_fgsm, y_test)
print(f'Accuracy on fgsm attack: {accuracy_adv_fgsm * 100:.2f}%')

loss_adv_hop_skip_jump, accuracy_adv_hop_skip_jump = model.evaluate(X_test_adv_hop_skip_jump, y_test)
print(f'Accuracy on hop skip jump attack: {accuracy_adv_hop_skip_jump * 100:.2f}%')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9798 - loss: 0.0483 
Accuracy on clean examples: 98.20%
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7691 - loss: 377.8260  
Accuracy on fgsm attack: 76.55%
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8243 - loss: 0.8233
Accuracy on hop skip jump attack: 80.95%


In [27]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
#y_pred_adv_cw = model.predict(X_test_adv_cw)
#y_pred_adv_cw = (y_pred_adv_cw > 0.5)
y_pred_adv_fgsm = model.predict(X_test_adv_fgsm)
y_pred_adv_fgsm = (y_pred_adv_fgsm > 0.5)
y_pred_adv_hop_skip_jump = model.predict(X_test_adv_hop_skip_jump)
y_pred_adv_hop_skip_jump = (y_pred_adv_hop_skip_jump > 0.5)

print("Normal Classification Report:")
print(classification_report(y_test, y_pred, target_names=['BENIGN', 'ATTACK']))
#print("C&W Classification Report:")
#print(classification_report(y_test, y_pred_adv_cw, target_names=['BENIGN', 'ATTACK']))
print("FGSM Classification Report:")
print(classification_report(y_test, y_pred_adv_fgsm, target_names=['BENIGN', 'ATTACK']))
print("Hop Skip Jump Classification Report:")
print(classification_report(y_test, y_pred_adv_hop_skip_jump, target_names=['BENIGN', 'ATTACK']))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 910us/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 792us/step
Normal Classification Report:
              precision    recall  f1-score   support

      BENIGN       0.99      0.99      0.99      1619
      ATTACK       0.94      0.97      0.95       381

   micro avg       0.98      0.98      0.98      2000
   macro avg       0.97      0.98      0.97      2000
weighted avg       0.98      0.98      0.98      2000
 samples avg       0.98      0.98      0.98      2000

FGSM Classification Report:
              precision    recall  f1-score   support

      BENIGN       0.80      0.94      0.87      1619
      ATTACK       0.08      0.02      0.04       381

   micro avg       0.77      0.77      0.77      2000
   macro avg       0.44      0.48      0.45      2000
weighted avg       0.67      0.77      0.71      2000
 samples avg       0.77      0.77      0.77      2000

Hop Skip Jump Classification Report:
              precisio

/home/felix/Documents/Uni/Master Arbeit/Python/xai-assisted-intrusion-detection-system/.env/lib64/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [29]:
# show accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# convert to binary 
y_test_binary = np.array(y_test).argmin(axis=1)
y_pred_binary = np.array(y_pred).argmin(axis=1)
y_pred_adv_fgsm_binary = np.array(y_pred_adv_fgsm).argmin(axis=1)
y_pred_adv_hop_skip_jump_binary = np.array(y_pred_adv_hop_skip_jump).argmin(axis=1)
y_pred_hop_skip_jump_binary = np.array(y_pred_adv_hop_skip_jump).argmin(axis=1)

print("Normal:")
print(f"Accuracy: {accuracy_score(y_test_binary, y_pred_binary)*100:.2f}%")
print(f"Precision: {precision_score(y_test_binary, y_pred_binary)*100:.2f}%")
print(f"Recall: {recall_score(y_test_binary, y_pred_binary)*100:.2f}%")
print(f"F1-Score: {f1_score(y_test_binary, y_pred_binary)*100:.2f}%")

print("FGSM:")
print(f"Accuracy: {accuracy_score(y_test_binary, y_pred_adv_fgsm_binary)*100:.2f}%")
print(f"Precision: {precision_score(y_test_binary, y_pred_adv_fgsm_binary)*100:.2f}%")
print(f"Recall: {recall_score(y_test_binary, y_pred_adv_fgsm_binary)*100:.2f}%")
print(f"F1-Score: {f1_score(y_test_binary, y_pred_adv_fgsm_binary)*100:.2f}%")

print("Hop Skip Jump:")
print(f"Accuracy: {accuracy_score(y_test_binary, y_pred_adv_hop_skip_jump_binary)*100:.2f}%")
print(f"Precision: {precision_score(y_test_binary, y_pred_adv_hop_skip_jump_binary)*100:.2f}%")
print(f"Recall: {recall_score(y_test_binary, y_pred_adv_hop_skip_jump_binary)*100:.2f}%")
print(f"F1-Score: {f1_score(y_test_binary, y_pred_adv_hop_skip_jump_binary)*100:.2f}%")

Normal:
Accuracy: 98.20%
Precision: 99.19%
Recall: 98.58%
F1-Score: 98.88%
FGSM:
Accuracy: 76.55%
Precision: 80.36%
Recall: 94.01%
F1-Score: 86.65%
Hop Skip Jump:
Accuracy: 80.95%
Precision: 80.95%
Recall: 100.00%
F1-Score: 89.47%


## Compare Data

In [30]:
#adv_cw_df = pd.DataFrame(X_test_adv_cw)
# set column names
#adv_cw_df.columns = X_test.columns
#print(adv_cw_df.head(2))

adv_fgsm_df = pd.DataFrame(X_test_adv_fgsm)
# set column names
adv_fgsm_df.columns = X_test.columns
print(adv_fgsm_df.head(2))

adv_hop_skip_jump_df = pd.DataFrame(X_test_adv_hop_skip_jump)
# set column names
adv_hop_skip_jump_df.columns = X_test.columns
print(adv_hop_skip_jump_df.head(2))

norm_df = pd.DataFrame(X_test_np)
print(norm_df.head(2))

    Destination Port   Flow Duration   Total Fwd Packets  \
0           0.085107             0.0                 0.0   
1           0.501221             0.0                 0.0   

    Total Backward Packets  Total Length of Fwd Packets  \
0                      0.0                          0.0   
1                      0.0                          0.5   

    Total Length of Bwd Packets   Fwd Packet Length Max  \
0                           0.0                0.500242   
1                           0.5                0.500000   

    Fwd Packet Length Min   Fwd Packet Length Mean   Fwd Packet Length Std  \
0                0.502581                  0.50101                     0.5   
1                0.500000                  0.50000                     0.5   

   ...   act_data_pkt_fwd   min_seg_size_forward  Active Mean   Active Std  \
0  ...                0.0                    1.0          0.0          0.0   
1  ...                0.0                    0.5          0.5          0

## Store Adversarial Instances

In [31]:
# store the adversarial examples
#adv_cw_df.to_csv("../CICIDS2017/adversarial_samples/X_test_small_adv_cw.csv", index=False)
adv_fgsm_df.to_csv("../CICIDS2017/adversarial_samples/X_test_adv_fgsm_proto.csv", index=False)
adv_hop_skip_jump_df.to_csv("../CICIDS2017/adversarial_samples/X_test_adv_hsj_proto.csv", index=False)